In [2]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv('D1.csv')
df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A']]

In [24]:
df.head()

,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A
0,26/08/16,Bayern Munich,Werder Bremen,H,1.10,10.00,23.00
1,27/08/16,Augsburg,Wolfsburg,A,2.80,3.25,2.60
2,27/08/16,Dortmund,Mainz,H,1.36,5.00,8.50
3,27/08/16,Ein Frankfurt,Schalke 04,H,3.40,3.30,2.20
4,27/08/16,FC Koln,Darmstadt,H,1.80,3.50,4.75


In [48]:
def count_matches(team, df):
    return df.groupby(df.HomeTeam).count()

In [49]:
count_matches('Bayern Munich', df)

,Date,AwayTeam,FTR,B365H,B365D,B365A
HomeTeam,,,,,,
Augsburg,17,17,17,17,17,17
Bayern Munich,17,17,17,17,17,17
Darmstadt,17,17,17,17,17,17
Dortmund,17,17,17,17,17,17
Ein Frankfurt,17,17,17,17,17,17
FC Koln,17,17,17,17,17,17
Freiburg,17,17,17,17,17,17
Hamburg,17,17,17,17,17,17
Hertha,17,17,17,17,17,17


In [50]:
import requests

page = requests.get('https://www.tipico.de/de/ergebnisse/fussball/deutschland/g42301-1-bundesliga/2008/kw5/')
page

<Response [200]>

In [54]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')

In [92]:
rows = soup.find_all('div', class_ = 'result')

In [98]:
import re

output = []
for row in rows:
    result = [re.sub(r'\t', '', re.sub(r'\n', '', div.get_text())) for div in row.find_all('div', class_ = 't_cell w_140 left')]
    result.extend([div.get_text() for div in row.find_all('div', class_ = 'qbut')])
    output.append(result)


df1 = pd.DataFrame(output, columns = ['HomeTeam', 'AwayTeam', 'TipicoH', 'TipicoD', 'TipicoA'])

In [249]:
def crawl_tipico_odds(year, kw):
    output = []
    url = 'https://www.tipico.de/de/ergebnisse/fussball/deutschland/g42301-1-bundesliga/' + str(year) + '/kw' + str(kw) + '/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows = soup.find_all('div', class_ = 'result')
    for row in rows:
        result = [re.sub(r'\t', '', re.sub(r'\n', '', div.get_text())).lower() for div in row.find_all('div', class_ = 't_cell w_140 left')]
        result.extend([div.get_text() for div in row.find_all('div', class_ = 'qbut')])
        output.append(result)
    
    return pd.DataFrame(output, columns = ['HomeTeam', 'AwayTeam', 'TipicoH', 'TipicoD', 'TipicoA'])

In [255]:
df_for_season = pd.DataFrame(columns = ['HomeTeam', 'AwayTeam', 'TipicoH', 'TipicoD', 'TipicoA'])
startyear = 2008
year = startyear
for kw in range(51 - 31):
    df_for_kw = crawl_tipico_odds(2009, kw + 32)
    df_for_season = df_for_season.append(df_for_kw).reset_index(drop = True)

for kw in range(18 - 1):
    df_for_kw = crawl_tipico_odds(2010, kw + 2)
    df_for_season = df_for_season.append(df_for_kw).reset_index(drop = True)

df_for_season.to_csv('tipico_odds_2009-2010.csv')

In [177]:
drop_rows = df_for_season.loc[df_for_season.TipicoH == '\xa0']
for row_index, row in drop_rows.iterrows():
    df_for_season = df_for_season.drop(row_index)

df_for_season.to_csv('tipico_odds_2007-2008.csv')

In [263]:
team_translation = {
    '1. fc köln': 'FC Koln',
    '1. fc nürnberg' : 'Nurnberg',
    '1.fc köln': 'FC Koln',
    '1.fc nürnberg': 'Nurnberg',
    '1899 hoffenheim': 'Hoffenheim',
    'armin. bielefeld': 'Bielefeld',
    'arminia bielefeld': 'Bielefeld',
    'bayer 04 leverkusen': 'Leverkusen',
    'bayer leverkusen': 'Leverkusen',
    'bayern münchen': 'Bayern Munich',
    'bor. dortmund': 'Dortmund',
    "bor. m'gladbach": "M'gladbach",
    'boru. dortmund': 'Dortmund',
    'borussia dortmund': 'Dortmund',
    'eintr. frankfurt': 'Ein Frankfurt',
    'eintracht frankfurt': 'Ein Frankfurt',
    'energie cottbus': 'Cottbus',
    'fc bayern münchen': 'Bayern Munich',
    'fc schalke 04': 'Schalke 04',
    'fsv mainz 05 ': 'Mainz',
    'hamburger sv': 'Hamburg',
    'hannover 96': 'Hannover',
    'hansa rostock': 'Hansa Rostock',
    'hertha berlin': 'Hertha',
    'hertha bsc': 'Hertha',
    'hertha bsc berlin': 'Hertha',
    'hoffenheim': 'Hoffenheim',
    'karlsruher sc': 'Karlsruhe',
    'msv duisburg': 'Duisburg',
    'sc freiburg': 'Freiburg',
    'schalke 04': 'Schalke 04',
    'sv werder bremen': 'Werder Bremen',
    'vfb stuttgart': 'Stuttgart',
    'vfl bochum': 'Bochum',
    'vfl wolfsburg': 'Wolfsburg',
    'werder bremen': 'Werder Bremen'
}

In [256]:
df = pd.read_csv('D1_2009-2010.csv')
df = df[['HomeTeam']]
df.groupby(df.HomeTeam).count()

""
HomeTeam
Bayern Munich
Bochum
Dortmund
Ein Frankfurt
FC Koln
Freiburg
Hamburg
Hannover
Hertha


In [257]:
df = pd.read_csv('tipico_odds_2009-2010.csv')
df = df[['HomeTeam']]
df.groupby(df.HomeTeam).count()

""
HomeTeam
1. fc köln
1. fc nürnberg
1.fc köln
1.fc nürnberg
bayer leverkusen
bayern münchen
bor. dortmund
bor. m'gladbach
borussia dortmund


In [264]:
df_for_season = pd.read_csv('tipico_odds_2009-2010.csv')
df_main = pd.read_csv('D1_2009-2010.csv')
df_main['TipicoH'] = pd.Series(np.zeros(df_for_season.shape[0]), index = df_for_season.index)
df_main['TipicoD'] = pd.Series(np.zeros(df_for_season.shape[0]), index = df_for_season.index)
df_main['TipicoA'] = pd.Series(np.zeros(df_for_season.shape[0]), index = df_for_season.index)
for row_index, row in df_for_season.iterrows():
    df_main.loc[(df_main.HomeTeam == team_translation[row['HomeTeam']]) & (df_main.AwayTeam == team_translation[row['AwayTeam']]), ['TipicoH', 'TipicoD', 'TipicoA']] = [row['TipicoH'], row['TipicoD'], row['TipicoA']]

df_main = df_main[['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'TipicoH', 'TipicoD', 'TipicoA']]
df_main.to_csv('tipico_odds_2009-2010.csv')